**Random Forest: 2011-2015 training set**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from datetime import datetime
from sklearn.linear_model import LogisticRegression
import math
import seaborn as sns
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [2]:
data=pd.read_csv("./final_project_data.csv")
data=data.dropna()
data.index=range(len(data))

In [5]:
rf= RandomForestClassifier(random_state = 99, class_weight= "balanced_subsample")
print(rf.get_params())

{'bootstrap': True, 'class_weight': 'balanced_subsample', 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 'warn', 'n_jobs': None, 'oob_score': False, 'random_state': 99, 'verbose': 0, 'warm_start': False}


In [6]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 45, stop = 105, num = 15)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(4, 8, num = 5)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

#Grid
random_grid = {'n_estimators': n_estimators,'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

{'n_estimators': [45, 49, 53, 57, 62, 66, 70, 75, 79, 83, 87, 92, 96, 100, 105], 'max_features': ['auto', 'sqrt'], 'max_depth': [4, 5, 6, 7, 8], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [7]:
#Experiment for the first training set: 2008-2012
i=2015
train_rf_4=data[(data.date<=int(str(i)+'1231'))&(data.date>=int(str(i-4)+'0101'))]
train_rf_4=train_rf_4.dropna()
X_train_rf_4=np.asarray(train_rf_4.loc[:,'ret_raw_norm_lag_21':'isJan'])
Y_train_rf_4=np.asarray(train_rf_4.loc[:,'target'])

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, cv = 5, verbose=2, random_state=42, n_jobs = -1, pre_dispatch=500)
rf_random.fit(X_train_rf_4, Y_train_rf_4)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  50 | elapsed: 144.7min remaining: 236.2min
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed: 289.1min remaining: 32.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 307.6min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators='warn', n_jobs=None, oob_score=False,
            random_state=99, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [45, 49, 53, 57, 62, 66, 70, 75, 79, 83, 87, 92, 96, 100, 105], 'max_features': ['auto', 'sqrt'], 'max_depth': [4, 5, 6, 7, 8], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
          pre_dispatch=500, random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [8]:
rf_random.best_params_

{'n_estimators': 79,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 8}

Checking accuracy

In [9]:
test_rf_4=data[(data.date<=int(str(i+1)+'1231'))&(data.date>=int(str(i+1)+'0101'))]
test_rf_4=test_rf_4.dropna()
X_test_rf_4=np.asarray(test_rf_4.loc[:,'ret_raw_norm_lag_21':'isJan'])
Y_test_rf_4=np.asarray(test_rf_4.loc[:,'target'])

acc_os=1.0*(rf_random.best_estimator_.predict(X_test_rf_4)==np.asarray(Y_test_rf_4)).sum()/len(Y_test_rf_4)
acc_in=1.0*(rf_random.best_estimator_.predict(X_train_rf_4)==np.asarray(Y_train_rf_4)).sum()/len(Y_train_rf_4)
print ('\033[1m' "Out of sample accuracy: {}".format((acc_os)*100))
print ('\033[1m' "In sample accuracy: {}".format((acc_in)*100))

Out of sample accuracy: 49.35522666913369
In sample accuracy: 54.03937625343941


** We need to run this**

In [10]:
X_train_,X_vali_,Y_train_,Y_vali_ = train_test_split(X_train_rf_4, Y_train_rf_4, test_size=0.1, random_state=99)

best_score = 0
n_estimators = [int(x) for x in np.linspace(start = 75, stop = 85, num = 7)]
max_depth = [int(x) for x in np.linspace(6, 9, num = 3)]
for n in n_estimators:
    for d in max_depth:
        clf =  RandomForestClassifier(random_state = 99, class_weight= "balanced_subsample", n_estimators= n,
        min_samples_split= 5, min_samples_leaf= 1, max_features='sqrt', max_depth=d, n_jobs=-1)
        clf.fit(X_train_,Y_train_) 
        score = clf.score(X_vali_, Y_vali_)
        if score > best_score:
            best_score = score
        print ("Score:", score, "n_estimator= ", n, "Max_depth=",d )

Score: 0.5254338201618238 n_estimator=  75 Max_depth= 6
Score: 0.5273217647956847 n_estimator=  75 Max_depth= 7
Score: 0.5321306803725001 n_estimator=  75 Max_depth= 9
Score: 0.5252353569792886 n_estimator=  76 Max_depth= 6
Score: 0.5271487456109104 n_estimator=  76 Max_depth= 7
Score: 0.5322375451630961 n_estimator=  76 Max_depth= 9
Score: 0.5253218665716758 n_estimator=  78 Max_depth= 6
Score: 0.5270978576153885 n_estimator=  78 Max_depth= 7
Score: 0.5323494987532441 n_estimator=  78 Max_depth= 9
Score: 0.525357488168541 n_estimator=  80 Max_depth= 6
Score: 0.5273217647956847 n_estimator=  80 Max_depth= 7
Score: 0.5322579003613047 n_estimator=  80 Max_depth= 9
Score: 0.5252506233779451 n_estimator=  81 Max_depth= 6
Score: 0.5270775024171798 n_estimator=  81 Max_depth= 7
Score: 0.5323087883568266 n_estimator=  81 Max_depth= 9
Score: 0.5250877817922752 n_estimator=  83 Max_depth= 6
Score: 0.5269502824283752 n_estimator=  83 Max_depth= 7
Score: 0.5325530507353315 n_estimator=  83 Max_de

In [5]:
clf =  RandomForestClassifier(random_state = 99, class_weight= "balanced_subsample", n_estimators=85,
min_samples_split= 5, min_samples_leaf= 1, max_features='sqrt', max_depth=9, n_jobs=-1)
clf.fit(X_train_rf_4,Y_train_rf_4) 

acc_os=1.0*(clf.predict(X_test_rf_4)==np.asarray(Y_test_rf_4)).sum()/len(X_test_rf_4)
acc_in=1.0*(clf.predict(X_train_rf_4)==np.asarray(Y_train_rf_4)).sum()/len(Y_train_rf_4)
print ('\033[1m' "Out of sample accuracy: {}".format((acc_os)*100))
print ('\033[1m' "In sample accuracy: {}".format((acc_in)*100))

Out of sample accuracy: 49.37102164136839
In sample accuracy: 54.92340566069901
